In [ ]:
!pip install groq
!pip install unidecode

!pip install -q pyserini==0.21.0

!pip install -q faiss-cpu
!pip install -q sentence_transformers

!pip install transformers
!pip install -q accelerate
!pip install -q bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 24.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 103.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 51.7 MB/s e

In [ ]:
from groq import Groq

import os
import re
import math
import json

import spacy
import pickle

from pyserini.search.lucene import LuceneSearcher
from collections import defaultdict
from tqdm import tqdm


In [ ]:
faq_path = 'dataset.json'
laws_path = 'dataset_8981.json'

In [ ]:
faq_set =  json.load(open(faq_path, 'r'))
laws_set = json.load(open(laws_path, 'r'))

In [ ]:
# Adaptado de Leandro Carísio

question_answer = []

for i in range(len(faq_set)):
  qa = faq_set[i]

  question = qa['question']
  answer = qa['answer']
  source = qa['source']

  question_answer.append({"question": question, "answer": answer.strip(), "source": source})
  print(f"{i}: {question}\n{answer}\n{source}")
  print('----------------------------------------------------------------------')


0: O que é operação day trade?
Considera-se day trade a operação ou a conjugação de operações iniciadas e encerradas em um mesmo dia, com o mesmo ativo, em uma mesma instituição intermediadora, em que a quantidade negociada tenha sido liquidada, total ou parcialmente. Na apuração do resultado da operação day trade são considerados, pela ordem, o primeiro negócio de compra com o primeiro de venda ou o primeiro negócio de venda com o primeiro de compra, sucessivamente. Será admitida a compensação de perdas incorridas em operações de day trade realizadas no mesmo dia.
Lei nº 9.959, de 27 de janeiro de 2000, art. 8º; Lei nº 12.350, de 20 de dezembro de 2010, art. 45
----------------------------------------------------------------------
1: Qual é a alíquota de incidência do IR aplicável nas operações realizadas em bolsa?
Os ganhos líquidos auferidos em operações realizadas em bolsas de valores, de mercadorias, de futuros, e assemelhadas, inclusive day trade, serão tributados às seguintes al

In [ ]:
documents = []
all_texts = []

for item in laws_set:
    for artigos in item["artigos"]:
        for paragrafos in artigos["paragrafos"]:
            for incisos in paragrafos["incisos"]:
                documents.append({
                    "lei":             item["titulo"],
                    "content":          artigos["artigo"] + " " + artigos["texto"] + " " +  paragrafos["paragrafo"] + " " +  paragrafos["texto"] + " " + incisos
                })

In [ ]:
for item in laws_set:
  for artigos in item["artigos"]:
    print(artigos["paragrafos"])

[{'paragrafo': '§ 1º', 'texto': 'O Ministério da Fazenda divulgará a expressão monetária da Ufir trimestral com base no IPCA - Série Especial de que trata o art. 2º daLei nº 8.383, de 30 de dezembro de 1991.', 'incisos': ['O Ministério da Fazenda divulgará a expressão monetária da Ufir trimestral com base no IPCA - Série Especial de que trata o art. 2º daLei nº 8.383, de 30 de dezembro de 1991.']}, {'paragrafo': '§ 2º', 'texto': 'O IPCA - Série Especial será apurado a partir do período de apuração iniciado em 16 de dezembro de 1994 e divulgado trimestralmente pela Fundação Instituto Brasileiro de Geografia e Estatística (FIBGE).', 'incisos': ['O IPCA - Série Especial será apurado a partir do período de apuração iniciado em 16 de dezembro de 1994 e divulgado trimestralmente pela Fundação Instituto Brasileiro de Geografia e Estatística (FIBGE).']}, {'paragrafo': '§ 3º', 'texto': 'A expressão monetária da Ufir referente ao primeiro trimestre de 1995 é de R$ 0,6767.', 'incisos': ['A expres

In [ ]:
print(documents[4])
len(documents)

{'lei': 'LEI Nº 8.981, DE 20 DE JANEIRO DE 1995', 'content': 'Art. 3º A base de cálculo e o Imposto de Renda das pessoas jurídicas tributadas com base no lucro real, presumido ou arbitrado, correspondentes aos períodos-base encerrados no ano-calendário de 1994, serão expressos em quantidade de Ufir, observada a legislação então vigente. § 1º A base de cálculo e o Imposto de Renda das pessoas jurídicas tributadas com base no lucro real, presumido ou arbitrado, correspondentes aos períodos-base encerrados no ano-calendário de 1994, serão expressos em quantidade de Ufir, observada a legislação então vigente. A base de cálculo e o Imposto de Renda das pessoas jurídicas tributadas com base no lucro real, presumido ou arbitrado, correspondentes aos períodos-base encerrados no ano-calendário de 1994, serão expressos em quantidade de Ufir, observada a legislação então vigente.'}


192

In [ ]:
nlp = spacy.blank("pt")
nlp.add_pipe("sentencizer")

stride = 2
max_length = 3

def window(documents, stride=2, max_length=3):
  treated_documents = []

  for j, document in enumerate(tqdm(documents)):
    doc_text = document['content']
    doc = nlp(doc_text)
    sentences = [sent.text.strip() for sent in doc.sents]
    for i in range(0, len(sentences), stride):
      segment = ' '.join(sentences[i:i + max_length])
      treated_documents.append({
          "title": document['lei'],
          "contents": document['lei']+". "+segment,
          "segment": segment
      })
      if i + max_length >= len(sentences):
        break
  return treated_documents


treated_documents = window(documents)

  0%|          | 0/192 [00:00<?, ?it/s]

In [ ]:
os.makedirs("outputs", exist_ok=True)

with open("outputs/output.jsonl", "w") as output_file:
  for i, document in enumerate(treated_documents):
      if i > 0:
          output_file.write("\n")

      document['id'] = i
      output_file.write(json.dumps(document))

In [ ]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input outputs \
  --index output \
  --language pt\
  --generator DefaultLuceneDocumentGenerator \
  --threads 1 \
  --storePositions --storeDocvectors --storeRaw

2024-06-27 00:45:42,072 INFO  [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO
2024-06-27 00:45:42,074 INFO  [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer...
2024-06-27 00:45:42,074 INFO  [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============
2024-06-27 00:45:42,074 INFO  [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: outputs
2024-06-27 00:45:42,075 INFO  [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection
2024-06-27 00:45:42,075 INFO  [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator
2024-06-27 00:45:42,075 INFO  [main] index.IndexCollection (IndexCollection.java:388) - Threads: 1
2024-06-27 00:45:42,076 INFO  [main] index.IndexCollection (IndexCollection.java:389) - Language: pt
2024-06-27 00:45:42,076 INFO  [main] index.IndexCollection (IndexCollectio

In [ ]:
## Trecho obtido na integra de Thiago Laitz

from math import exp
from typing import List

import torch
from tqdm.auto import tqdm
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BatchEncoding,
    AutoModelForSeq2SeqLM
)

class MonoT5():
    def __init__(self, model_name_or_path: str, fp8: bool = False):
        """
        Loads the T5 model from the given path.
        Args:
            model_name_or_path: path to the model
            fp8: whether the model should be loaded using FP8
        """
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
        # The training was carried out using two specific tokens for relevant and non-relevant passages
        self.token_false_id = self.tokenizer.get_vocab()['▁false']
        self.token_true_id  = self.tokenizer.get_vocab()['▁true']

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Loads the model with model_args
        model_args = {}
        if fp8:
            model_args["torch_dtype"] = torch.float16
            model_args["load_in_8bit"] = True

        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path, **model_args)

    @torch.no_grad()
    def rescore(self, query: str, batch: List[str]):
        """
        Adapted from Pygaggle's repo.
        Rescore all documents for the given query.
        Args:
            query: the query for ranking
            batch: list of passages for ranking
        """
        scores = []
        # Creates the inputs to the model
        queries_documents = [f"Query: {query} Document: {doc} Relevant:" for doc in batch]
        tokenized = self.tokenizer(
            queries_documents,
            padding=True,
            truncation="longest_first",
            return_tensors="pt",
            max_length=512,
        ).to(self.device)
        input_ids = tokenized["input_ids"].to(self.device)
        attention_mask = tokenized["attention_mask"].to(self.device)
        _ , batch_scores = self.greedy_decode(model=self.model,
                                            input_ids=input_ids,
                                            length=1,
                                            attention_mask=attention_mask,
                                            return_last_logits=True)
        batch_scores = batch_scores[:, [self.token_false_id, self.token_true_id]]
        batch_scores = torch.nn.functional.log_softmax(batch_scores, dim=1)
        batch_log_probs = batch_scores[:, 1].tolist()
        batch_probs = [exp(log_prob) for log_prob in batch_log_probs]
        scores.extend(batch_probs)
        return scores

    @torch.no_grad()
    def greedy_decode(
        self,
        model,
        input_ids: torch.Tensor,
        length: int,
        attention_mask: torch.Tensor = None,
        return_last_logits: bool = True
    ):

        decode_ids = torch.full((input_ids.size(0), 1),
                                model.config.decoder_start_token_id,
                                dtype=torch.long).to(input_ids.device)
        encoder_outputs = model.get_encoder()(input_ids, attention_mask=attention_mask)
        next_token_logits = None
        for _ in range(length):
            model_inputs = model.prepare_inputs_for_generation(
                decode_ids,
                encoder_outputs=encoder_outputs,
                past=None,
                attention_mask=attention_mask,
                use_cache=True)
            outputs = model(**model_inputs)  # (batch_size, cur_len, vocab_size)
            next_token_logits = outputs[0][:, -1, :]  # (batch_size, vocab_size)
            decode_ids = torch.cat([decode_ids,
                                    next_token_logits.max(1)[1].unsqueeze(-1)],
                                dim=-1)
        if return_last_logits:
            return decode_ids, next_token_logits
        return decode_ids

In [ ]:
t5_ranker = MonoT5('castorini/monot5-large-msmarco-10k', fp8=True)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [ ]:
searcher = LuceneSearcher('/content/output')

def get_passages(query, top_k):
    hits = searcher.search(query, top_k)
    return hits

In [ ]:
## Adaptado de Thiago Laitz

import requests
import time

def get_llm_response(prompt: str, system_prompt: str = None):
    for _ in range(15):
        data = {
            "model": "llama3-70b-8192",
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0,
            "top_p": 1
        }
        if system_prompt:
            data["messages"].insert(0, {"role": "system", "content": system_prompt})
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer [KEY]"
        }

        response = requests.post(
            "https://api.groq.com/openai/v1/chat/completions",
            json=data,
            headers=headers
        )

        if not response.ok:
            time.sleep(3)
            continue

        response = response.json()

        return response["choices"][0]["message"]["content"].strip().lower()


In [ ]:
system_prompt = """
Você é um legislador e receberá informações especificas sobre leis, como por exemplo o nome da lei, seus artigos, paragrafos e incisos.
Você precisará responder a perguntas se baseado inteiramente nas informações fornecidas por prompt, isto é, você não pode utilizar seu conhecimento prévio para responder.
Caso os trechos das leis que você receber não forem o suficiente para responder a pergunta, informe que você não pode responder, já que qualquer resposta neste caso é incerta.
""".strip()

In [ ]:
user_prompt = r"""
Dada as seguintes informações sobre leis, tente responder a pergunta exclusivamente se baseando nestas informações. Responda em português. Após a resposta, inclua a fonte da informação (qual lei, paragrafo, inciso, entre outros).
Questão: {query}
""".strip()

In [ ]:
def get_search_answer(
    query: str,
    top_k_bm25: int = 50,
    top_k_ranker: int = 5
) -> tuple[str, float]:
    hits = searcher.search(query, top_k_bm25)
    paragraphs = [json.loads(hit.raw) for hit in hits]

    scores = t5_ranker.rescore(query, [p["contents"] for p in paragraphs])
    scores_paragraphs = list(zip(paragraphs, scores))
    sorted_paragraphs = sorted(scores_paragraphs, key=lambda x: x[1], reverse=True)
    documents_string = ""
    for idx, (doc, _) in enumerate(sorted_paragraphs[:top_k_ranker]):
        documents_string += f"Documento {idx} - {doc['title']}:\n{doc['contents']}\n\n"
    documents_string = documents_string.replace("{", "{{").replace("}", "}}")
    return documents_string

In [ ]:
#question = "Qual é a alíquota de incidência do IR aplicável nas operações realizadas em bolsa?"
#question = "Quais operações em bolsa são isentas do IR?"
question = question_answer[11]["question"]
print(question)


Qual é o tratamento tributário dos rendimentos produzidos por conta de depósito de poupança?


In [ ]:
answer = question_answer[11]["answer"]
source = question_answer[11]["source"]
print(answer)
print(source)

Os rendimentos obtidos em conta de depósito de poupança pela pessoa física estão isentos do imposto sobre a renda, ainda que em virtude de decisão judicial que tenha determinado a correção dos valores depositados por índice diferente do fixado pela autoridade monetária.
Lei nº 8.981, de 20 de janeiro de 1995, art. 68, inciso III


In [ ]:
content = user_prompt.format(query=question)
print(content)

Dada as seguintes informações sobre leis, tente responder a pergunta exclusivamente se baseando nestas informações. Responda em português. Após a resposta, inclua a fonte da informação (qual lei, paragrafo, inciso, entre outros).
Questão: Qual é o tratamento tributário dos rendimentos produzidos por conta de depósito de poupança?


In [ ]:
search_answer = get_search_answer(content)
print(search_answer)

Documento 0 - LEI Nº 8.981, DE 20 DE JANEIRO DE 1995:
LEI Nº 8.981, DE 20 DE JANEIRO DE 1995. Art. 7º A partir de 1º de janeiro de 1995, a renda e os proventos de qualquer natureza, inclusive os rendimentos e ganhos de capital, percebidos por pessoas físicas residentes ou domiciliadas no Brasil, serão tributados pelo Imposto de Renda na forma da legislação vigente, com as modificações introduzidas por esta lei. SEÇÃO II
Da Incidência Mensal do Imposto
SEÇÃO III
Da Declaração de Rendimentos
SEÇÃO IV
Tributação dos Ganhos de Capital das Pessoas Físicas
Art. 21. O ganho de capital percebido por pessoa física em decorrência da alienação de bens e direitos de qualquer natureza sujeita-se à incidência do imposto sobre a renda, com as seguintes alíquotas:(Redação dada pela Lei nº 13.259, de 2016)Produção de efeito
I - 15% (quinze por cento) sobre a parcela dos ganhos que não ultrapassar R$ 5.000.000,00 (cinco milhões de reais);(Redação dada pela Lei nº 13.259, de 2016)Produção de efeito
II - 

In [ ]:
user_prompt += "\nDocumentos: {documents}\nAnswer:"
print(user_prompt)

Dada as seguintes informações sobre leis, tente responder a pergunta exclusivamente se baseando nestas informações. Responda em português. Após a resposta, inclua a fonte da informação (qual lei, paragrafo, inciso, entre outros).
Questão: {query}
Documentos: {documents}
Answer:


In [ ]:
content = user_prompt.format(documents = search_answer, query=question)
print(content)

Dada as seguintes informações sobre leis, tente responder a pergunta exclusivamente se baseando nestas informações. Responda em português. Após a resposta, inclua a fonte da informação (qual lei, paragrafo, inciso, entre outros).
Questão: Qual é o tratamento tributário dos rendimentos produzidos por conta de depósito de poupança?
Documentos: Documento 0 - LEI Nº 8.981, DE 20 DE JANEIRO DE 1995:
LEI Nº 8.981, DE 20 DE JANEIRO DE 1995. Art. 7º A partir de 1º de janeiro de 1995, a renda e os proventos de qualquer natureza, inclusive os rendimentos e ganhos de capital, percebidos por pessoas físicas residentes ou domiciliadas no Brasil, serão tributados pelo Imposto de Renda na forma da legislação vigente, com as modificações introduzidas por esta lei. SEÇÃO II
Da Incidência Mensal do Imposto
SEÇÃO III
Da Declaração de Rendimentos
SEÇÃO IV
Tributação dos Ganhos de Capital das Pessoas Físicas
Art. 21. O ganho de capital percebido por pessoa física em decorrência da alienação de bens e direi

In [ ]:
llm_answer = get_llm_response(content, system_prompt)

print(llm_answer)


não é possível responder a essa pergunta com base nas informações fornecidas, pois não há menção específica ao tratamento tributário dos rendimentos produzidos por conta de depósito de poupança. as informações fornecidas tratam de imposto de renda, ganhos de capital, e incidência do imposto sobre a renda, mas não abordam especificamente o tratamento tributário dos rendimentos de depósito de poupança.

fonte: não há fonte específica para essa resposta, pois não há informação relevante nos documentos fornecidos.
